In [9]:
import unittest
from freezegun import freeze_time
import pandas as pd
from datetime import datetime, timedelta

class TestWebAutomation(unittest.TestCase):
    def setUp(self):
        # Configuración inicial de prueba
        self.holidays = [
            "2024-01-01", "2024-01-15", "2024-02-19", "2024-05-27",
            "2024-07-04", "2024-09-02", "2024-10-14", "2024-11-11",
            "2024-11-28", "2024-12-25"
        ]

        self.config_with_business_days = {
            "use_business_day": True,
            "holidays": self.holidays
        }

        self.config_without_business_days = {
            "use_business_day": False,
            "holidays": self.holidays
        }

    def get_previous_day(self, config):
        holidays = config.get('holidays', [])
        use_business_day = config.get('use_business_day', False)

        if use_business_day:
            custom_bd = pd.offsets.CustomBusinessDay(holidays=holidays)
            previous_business_day = pd.Timestamp(datetime.now()) - custom_bd
            return previous_business_day.to_pydatetime()
        else:
            previous_day = datetime.now() - timedelta(days=1)
            return previous_day

    @freeze_time("2024-07-05")
    def test_previous_business_day_regular_day(self):
        # Configurado para el 5 de Julio de 2024 (después del 4 de Julio)
        previous_day = self.get_previous_day(self.config_with_business_days)
        self.assertEqual(previous_day.date(), datetime(2024, 7, 3).date())

    @freeze_time("2024-09-02")
    def test_previous_business_day_after_weekend(self):
        # Configurado para el 2 de Septiembre de 2024 (Lunes después del fin de semana y día laboral)
        previous_day = self.get_previous_day(self.config_with_business_days)
        self.assertEqual(previous_day.date(), datetime(2024, 8, 30).date())

    @freeze_time("2024-11-12")
    def test_previous_business_day_holiday(self):
        # Configurado para el 12 de Noviembre de 2024 (después del día del Veterano)
        previous_day = self.get_previous_day(self.config_with_business_days)
        self.assertEqual(previous_day.date(), datetime(2024, 11, 8).date())  # 11 de Nov es sábado, se observa el 10

    @freeze_time("2024-07-05")
    def test_previous_calendar_day(self):
        # Configurado para el 5 de Julio de 2024
        previous_day = self.get_previous_day(self.config_without_business_days)
        self.assertEqual(previous_day.date(), datetime(2024, 7, 4).date())

# Ejecutar las pruebas en Colab
unittest.main(argv=[''], exit=False)


....
----------------------------------------------------------------------
Ran 4 tests in 0.283s

OK


In [8]:
!pip install freezegun

In [3]:
unittest.main()


E
ERROR: /root/ (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/root/'

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (errors=1)


SystemExit: True

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
